In [ ]:
!pip install transformers

import tensorflow as tf
import numpy as np
import sklearn
from sklearn.metrics import precision_score,balanced_accuracy_score,recall_score,f1_score
import transformers
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import json
#import matplotlib.pyplot as plt
import random
#import seaborn as sn

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install newscatcher

In [4]:
from newscatcher import Newscatcher, describe_url
import re
from datetime import datetime
import pandas as pd

summary = []
published = []
title = []
#author = []
def fetch_news():
    websites = ['nytimes.com', 'theguardian.com'] #yahoo.com
    for i in websites:
      nyt = Newscatcher(website = i)
      results = nyt.get_news()
      articles = results['articles']
      for article in articles:
        summary.append(article['summary'])
        published.append(article['published'])
        title.append(article['title'])
        #author.append(article['author'])

##
fetch_news()


fmt = '%d%b%Y%H:%M:%S'
pat = r'\b[G]\w+'
pub_tim = []
def pub_time():
  
  for i in published:
    x1 = re.sub(pat,'',i)
    x2 = x1.split(',')
    x3 = x2[1].split('+')
    x4 = x3[0]
    x4 = x4.replace(' ','')
    split_dt = datetime.strptime(x4, fmt)
    pub_tim.append(split_dt)

##
pub_time()
now_time = datetime.now()

time_diff = []
def recency():
  for i in range(0,len(pub_tim)):
    diff = now_time - pub_tim[i]
    diff = int(diff.total_seconds() / 60)
    time_diff.append(diff)

##
recency()

In [5]:
clean_txt = []
def clean_content():
  for j in summary:
    a = re.sub(r"(<[A-Za-z0-9]+)|[^a-zA-Z0-9]"," ", str(j))
    clean_txt.append(a)

##
clean_content()

In [6]:
categories = np.load('/content/drive/MyDrive/categories.npy')
modelname = "bert-base-cased"
model = TFAutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/saved_pretrained/')

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/saved_pretrained/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [7]:
def sample_txt(txt,modelname,model):
  tokenizer = AutoTokenizer.from_pretrained(modelname)
  tokens = tokenizer(txt, padding=True,truncation=True, return_tensors='tf')
  logits=model.predict(dict(tokens), verbose=1).logits
  prob=tf.nn.softmax(logits, axis=1).numpy()
  pred=np.argmax(prob, axis=1)
  return categories[pred[0]]

In [8]:
all_cat = []
for i in clean_txt:
  cat = sample_txt(i,modelname,model)
  all_cat.append(cat)


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

1/1 [==============================] - 1s 662ms/step


In [9]:
cat_count = []
def impact():
  for i in all_cat:
    a = all_cat.count(i)
    cat_count.append(a)

##
impact()

In [10]:
df_impact = pd.DataFrame()
df_impact['Content'] = summary
#df_impact['Author'] = author
df_impact['Title'] = title
df_impact['categories'] = all_cat
df_impact['Age'] = time_diff
df_impact['Count'] = cat_count
df_impact['inf'] = df_impact["Count"].rank()


In [11]:
df_impact

,Content,Title,categories,Age,Count,inf
0,The country’s teachers are doing their best to...,War in Ukraine Disrupts Education of Millions ...,COLLEGE & EDUCATION,89,1,2.5
1,"Since their country invaded Ukraine, Russian t...",Russian Tech Industry Faces ‘Brain Drain’ as W...,WORLD NEWS,90,14,56.5
2,,"Frank James was quiet and gruff, his neighbors...",ENTERTAINMENT & FUN,182,16,86.5
3,The attack on Tuesday brought to life a horror...,Videos Show an Ordinary New York Morning Erupt...,CRIME,589,2,7.5
4,The fear in China is that the strict coronavir...,China’s ‘Zero-Covid’ Mess Proves Autocracy Hur...,WORLD NEWS,90,14,56.5
...,...,...,...,...,...,...
97,"<p>AI algorithms prompt robot to interrogate, ...",‘Mind-blowing’: Ai-Da becomes first robot to p...,ARTS & CULTURE,12535,5,17.0
98,<p>Run out of cyan? Help could be at hand as t...,A printer cartridge and chips please: why Deli...,WELLNESS & HEALTHY LIVING,21419,16,86.5
99,<p>The technology behind autonomous vehicles h...,How self-driving cars got stuck in the slow lane,BUSINESS & MONEY,24330,12,36.5
100,<p>He created the image file format that defin...,‘He helped shape the modern world’: gif invent...,ENTERTAINMENT & FUN,28110,16,86.5


In [12]:
df_impact = df_impact.drop('Count',axis = 1)

In [13]:
df_impact.to_csv('/content/drive/MyDrive/news_output.csv',index=False)

In [14]:
sorted_df = df_impact.sort_values(by=['Age', 'inf'])

In [15]:
sorted_df.head(10)

,Content,Title,categories,Age,inf
32,Ferdinand Marcos Jr. has spent his political c...,"Ahead of Philippines Election, Ferdinand Marco...",WORLD NEWS,11,56.5
70,<p>Schools are encouraging students to use men...,Young and depressed? Try Woebot! The rise of m...,WELLNESS & HEALTHY LIVING,29,86.5
6,The firm let consultants advise both drugmaker...,McKinsey Opened a Door in Its Firewall Between...,WELLNESS & HEALTHY LIVING,30,86.5
0,The country’s teachers are doing their best to...,War in Ukraine Disrupts Education of Millions ...,COLLEGE & EDUCATION,89,2.5
5,Research on past conflicts suggests that the w...,How Nature Becomes a Casualty of War,ENVIRONMENT & GREEN,89,12.5
11,The law professor Amy Gajda writes about the t...,‘Seek and Hide’ Grapples With the Complexity o...,POLITICS,90,25.0
16,Bret Stephens and Zack Beauchamp debate the bi...,The Dangerous Lesson Viktor Orban Taught Repub...,POLITICS,90,25.0
17,Ordinary voters are supposed to punish authori...,Trump Poses a Test Democracy Is Failing,POLITICS,90,25.0
19,What did it take for him to finally speak out?...,John Roberts Has Lost Control of the Supreme C...,POLITICS,90,25.0
1,"Since their country invaded Ukraine, Russian t...",Russian Tech Industry Faces ‘Brain Drain’ as W...,WORLD NEWS,90,56.5
